In [1]:
import sys
from datetime import datetime, timedelta
from difflib import get_close_matches
from collections import Counter
import numpy as np 
import pandas as pd 
import geopandas as gpd
from dateutil.parser import parse
from utils_covid import fix_country_names, get_names

In [2]:
# dates to use in graphics
today = datetime.utcnow().date()
yesterday = today - timedelta(days=1)
ontem = yesterday.strftime('%d/%m/%y')
ontem

'14/04/21'

#### Downloading the data
The world Covid-19 data is downloaded from CSSEGISandData repository from Jhon Hopkins University. The source is located at: https://github.com/CSSEGISandData

In [3]:
# lambda function to convert negative values to zero
func = lambda x: 0 if x < 0 else x

In [4]:
url1 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url3 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

In [5]:
# global data
df_confirmed = pd.read_csv(url1)
df_fatalities = pd.read_csv(url2)
df_recoveries = pd.read_csv(url3)

In [6]:
df_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Columns: 453 entries, Province/State to 4/14/21
dtypes: float64(2), int64(449), object(2)
memory usage: 969.8+ KB


In [7]:
df_confirmed.shape

(274, 453)

In [8]:
df_fatalities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Columns: 453 entries, Province/State to 4/14/21
dtypes: float64(2), int64(449), object(2)
memory usage: 969.8+ KB


In [9]:
df_fatalities.shape

(274, 453)

In [10]:
df_recoveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Columns: 453 entries, Province/State to 4/14/21
dtypes: float64(2), int64(449), object(2)
memory usage: 916.7+ KB


In [11]:
df_recoveries.shape

(259, 453)

#### Data munging

In [12]:
# cheking  if all columns are have the same name
dfs = [df_confirmed, df_fatalities, df_recoveries]

all([len(dfs[0].columns.intersection(df.columns)) == dfs[0].shape[1] for 
     df in dfs])

True

In [13]:
def get_large_to_long_format_JHU_data(df, col_values='Confirmed'):
    dates = df.columns[4:]
    df = df.melt(id_vars=['Province/State',
                          'Country/Region', 
                          'Lat', 
                          'Long'],
                 value_vars=dates, 
                 var_name='Date', 
                 value_name=col_values)
    return df

In [14]:
confirmed = get_large_to_long_format_JHU_data(df_confirmed, col_values='Confirmed')
deaths = get_large_to_long_format_JHU_data(df_fatalities, col_values='Deaths')
recovered = get_large_to_long_format_JHU_data(df_recoveries, col_values='Recovered')

In [15]:
confirmed.shape, deaths.shape, recovered.shape

((123026, 6), (123026, 6), (116291, 6))

In [16]:
def merge_JHU_data_frames(df1, df2, df3):
    full_df = pd.merge(left=df1, right=df2, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
    full_df = pd.merge(left=full_df, right=df3, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
    return full_df

In [17]:
world_covid = merge_JHU_data_frames(confirmed, deaths, recovered)

In [18]:
# removing county wise data to avoid double counting
world_covid = world_covid[world_covid['Province/State'].str.contains(',')!=True]

In [19]:
def cleaning_data(df):
    # dates string to date datetime
    df['Date'] = pd.to_datetime(df['Date'])
    # filling the missing values
    df['Recovered'] = df['Recovered'].fillna(0)
    # changing data types floats ---> integer
    df['Recovered'] = df['Recovered'].astype('int')
    # removing canada's recovered values
    df = df[df['Province/State'].str.contains('Recovered')!=True]
    # removing county wise data to avoid double counting
    df = df[df['Province/State'].str.contains(',')!=True]
    # the active cases are the actual covid cases
    df['Active'] = df['Confirmed'] - (df['Deaths'] + df['Recovered'])
    # fill missing province/state value with ''
    df[['Province/State']] = df[['Province/State']].fillna('')
    # fill missing numerical values with 0
    cols = ['Confirmed', 'Deaths', 'Recovered', 'Active']
    df[cols] = df[cols].fillna(0)
    # fixing datatypes
    df['Recovered'] = df['Recovered'].astype(int)
    # cruise ship rows containing COVID-19 reported cases
    cruise = df['Province/State'].str.contains('Grand Princess') | \
    df['Province/State'].str.contains('Diamond Princess') | \
    df['Country/Region'].str.contains('Diamond Princess') | df['Country/Region'].str.contains('MS Zaandam')
    # ship
    ship = df[cruise]
    # Latest cases from the ships
    ship_latest = ship[ship['Date'] == max(ship['Date'])]
    # skipping rows with ships info
    df = df[~(cruise)]
    # fixing Country values
    df.loc[world_covid['Province/State'] == 'Greenland', 'Country/Region'] = 'Greenland'
    # replacing Mainland china with just China
    df['Country/Region'] = df['Country/Region'].replace('Mainland China', 'China')
    return df

In [20]:
world_covid_clean = cleaning_data(world_covid)

In [21]:
world_covid_clean

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0
1,,Albania,41.153300,20.168300,2020-01-22,0,0,0,0
2,,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0
3,,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0
4,,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0
...,...,...,...,...,...,...,...,...,...
123021,,Vietnam,14.058324,108.277199,2021-04-14,2733,35,2445,253
123022,,West Bank and Gaza,31.952200,35.233200,2021-04-14,274690,2923,239291,32476
123023,,Yemen,15.552727,48.516388,2021-04-14,5582,1083,2128,2371
123024,,Zambia,-13.133897,27.849332,2021-04-14,90389,1229,88241,919


In [22]:
world_covid.values.base is world_covid_clean.values.base

False

In [23]:
def test_cleaning(df):
    # test if the cruise data was clean
    assert df['Province/State'].str.contains('Grand Princess').sum() == 0
    # check if the Recover column is type integer
    assert df['Recovered'].dtype == int
    # check if the date data is datetime type
    assert np.issubdtype(world_covid_clean['Date'].dtype, np.datetime64) == True
    # checks if the Mainland China was modified to China
    assert df[df['Country/Region']  == 'Mainland China'].empty == True
    # check if the new columns Active is data type integer
    df['Active'].dtype  == int

In [24]:
test_cleaning(world_covid_clean)

In [25]:
world_covid_clean.shape # orld_covid (121856, 8)

(120332, 9)

In [26]:
121856 - 120064

1792

In [27]:
world_covid_clean.rename(columns={'Country/Region':'country',
                                  'Province/State':'province',
                                  'Lat': 'lat',
                                  'Long': 'lon',
                                  'Date': 'date',
                                  'Confirmed': 'confirmed',
                                  'Deaths': 'deaths',
                                  'Recovered': 'recovered',
                                  'Active': 'active'                               
                                 }, inplace=True)

In [28]:
world_covid_clean

,province,country,lat,lon,date,confirmed,deaths,recovered,active
0,,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0
1,,Albania,41.153300,20.168300,2020-01-22,0,0,0,0
2,,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0
3,,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0
4,,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0
...,...,...,...,...,...,...,...,...,...
123021,,Vietnam,14.058324,108.277199,2021-04-14,2733,35,2445,253
123022,,West Bank and Gaza,31.952200,35.233200,2021-04-14,274690,2923,239291,32476
123023,,Yemen,15.552727,48.516388,2021-04-14,5582,1083,2128,2371
123024,,Zambia,-13.133897,27.849332,2021-04-14,90389,1229,88241,919


In [29]:
world_covid_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120332 entries, 0 to 123025
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   province   120332 non-null  object        
 1   country    120332 non-null  object        
 2   lat        119883 non-null  float64       
 3   lon        119883 non-null  float64       
 4   date       120332 non-null  datetime64[ns]
 5   confirmed  120332 non-null  int64         
 6   deaths     120332 non-null  int64         
 7   recovered  120332 non-null  int64         
 8   active     120332 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 9.2+ MB


In [30]:
world_covid_clean.isnull().sum()

province       0
country        0
lat          449
lon          449
date           0
confirmed      0
deaths         0
recovered      0
active         0
dtype: int64

In [31]:
# checking for funny names
get_names(world_covid_clean, 'country')

Afghanistan                   Albania                       Algeria
Andorra                       Angola                        Antigua and Barbuda
Argentina                     Armenia                       Australia
Austria                       Azerbaijan                    Bahamas
Bahrain                       Bangladesh                    Barbados
Belarus                       Belgium                       Belize
Benin                         Bhutan                        Bolivia
Bosnia and Herzegovina        Botswana                      Brazil
Brunei                        Bulgaria                      Burkina Faso
Burma                         Burundi                       Cabo Verde
Cambodia                      Cameroon                      Canada
Central African Republic      Chad                          Chile
China                         Colombia                      Comoros
Congo (Brazzaville)           Congo (Kinshasa)              Costa Rica
Cote d'Ivoire              

In [32]:
def fix_country_names(df):
    '''
    Cleaning up after data management 
    '''
    # Asian Countries
    df['country'] = np.where((df['country']  == 'Korea, South'),'South Korea', df['country'])
    df['country'] = np.where((df['country']  == 'Taiwan*'),'Taiwan', df['country'])
    df['country'] = np.where((df['country']  == 'West Bank and Gaza'),'Cisjordania', df['country'])
    #European Countries
    df['country'] = np.where((df['country']  == 'Bosnia and Herzegovina'),'Bosnia', df['country'])
    # others
    df['country'] = np.where((df['country']  == 'Australian Capital Territory'), 'Australia', df['country'])
    df['country'] = np.where((df['country']  == 'US'), 'United States of America', df['country'])
    #African Countries
    df['country'] = np.where((df['country']  == 'Congo (Brazzaville)'),'Congo', df['country'])
    df['country'] = np.where((df['country']  == 'Congo (Kinshasa)'),'Congo', df['country'])
    return df

In [33]:
fix_country_names(world_covid_clean)

,province,country,lat,lon,date,confirmed,deaths,recovered,active
0,,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0
1,,Albania,41.153300,20.168300,2020-01-22,0,0,0,0
2,,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0
3,,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0
4,,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0
...,...,...,...,...,...,...,...,...,...
123021,,Vietnam,14.058324,108.277199,2021-04-14,2733,35,2445,253
123022,,Cisjordania,31.952200,35.233200,2021-04-14,274690,2923,239291,32476
123023,,Yemen,15.552727,48.516388,2021-04-14,5582,1083,2128,2371
123024,,Zambia,-13.133897,27.849332,2021-04-14,90389,1229,88241,919


In [34]:
# testing the alterations
world_covid_clean[world_covid_clean['country'] == 'US']

,province,country,lat,lon,date,confirmed,deaths,recovered,active


In [35]:
world_covid_clean[world_covid_clean['country'] == 'Congo (Brazzaville)']

,province,country,lat,lon,date,confirmed,deaths,recovered,active


In [36]:
world_covid_clean[world_covid_clean['country'] == 'Congo']

,province,country,lat,lon,date,confirmed,deaths,recovered,active
93,,Congo,-0.2280,15.8277,2020-01-22,0,0,0,0
94,,Congo,-4.0383,21.7587,2020-01-22,0,0,0,0
367,,Congo,-0.2280,15.8277,2020-01-23,0,0,0,0
368,,Congo,-4.0383,21.7587,2020-01-23,0,0,0,0
641,,Congo,-0.2280,15.8277,2020-01-24,0,0,0,0
...,...,...,...,...,...,...,...,...,...
122298,,Congo,-4.0383,21.7587,2021-04-12,28542,745,25841,1956
122571,,Congo,-0.2280,15.8277,2021-04-13,10084,137,8208,1739
122572,,Congo,-4.0383,21.7587,2021-04-13,28665,745,25841,2079
122845,,Congo,-0.2280,15.8277,2021-04-14,10084,139,8208,1737


In [37]:
world_covid_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120332 entries, 0 to 123025
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   province   120332 non-null  object        
 1   country    120332 non-null  object        
 2   lat        119883 non-null  float64       
 3   lon        119883 non-null  float64       
 4   date       120332 non-null  datetime64[ns]
 5   confirmed  120332 non-null  int64         
 6   deaths     120332 non-null  int64         
 7   recovered  120332 non-null  int64         
 8   active     120332 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 9.2+ MB


In [38]:
world_covid_clean.shape

(120332, 9)

In [39]:
# cheking some examples of negative values
print(world_covid_clean['confirmed'].min())
print(world_covid_clean['deaths'].min())
print(world_covid_clean['recovered'].min())
print(world_covid_clean['active'].min())

0
0
0
-54890


In [40]:
world_covid_clean['active'] = world_covid_clean['active'].apply(func)

In [41]:
# cheking some examples of negative values
print(world_covid_clean['confirmed'].min())
print(world_covid_clean['deaths'].min())
print(world_covid_clean['recovered'].min())
print(world_covid_clean['active'].min())

0
0
0
0


In [42]:
# save data
world_covid_clean.to_csv(f'Data/New/world_covid_clean_{yesterday}.csv', index=False)

#### Grouping data

In [43]:
world_covid_clean.head()

,province,country,lat,lon,date,confirmed,deaths,recovered,active
0,,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0
1,,Albania,41.15330,20.168300,2020-01-22,0,0,0,0
2,,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0
3,,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0
4,,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0


In [44]:
# Grouping by date and countries
world_covid_grouped = world_covid_clean.groupby(['date',
                                                 'country'])[['confirmed', 
                                                              'deaths', 
                                                              'recovered', 
                                                              'active']].sum().reset_index()


In [45]:
world_covid_clean.values.base is world_covid_grouped.values.base

False

In [46]:
# obtaining the daily cases
new_cases = world_covid_grouped.groupby(['country', 
                                         'date', ])[['confirmed', 
                                                     'deaths', 
                                                     'recovered']].sum().diff().reset_index()

In [47]:
# masking where there are differences
fil  = new_cases['country'] != new_cases['country'].shift(1)

# applying the mask to fill with nan
new_cases.loc[fil, 'confirmed'] = np.nan
new_cases.loc[fil, 'deaths'] = np.nan
new_cases.loc[fil, 'recovered'] = np.nan

new_cases

,country,date,confirmed,deaths,recovered
0,Afghanistan,2020-01-22,NaN,NaN,NaN
1,Afghanistan,2020-01-23,0.0,0.0,0.0
2,Afghanistan,2020-01-24,0.0,0.0,0.0
3,Afghanistan,2020-01-25,0.0,0.0,0.0
4,Afghanistan,2020-01-26,0.0,0.0,0.0
...,...,...,...,...,...
85305,Zimbabwe,2021-04-10,126.0,3.0,31.0
85306,Zimbabwe,2021-04-11,15.0,0.0,11.0
85307,Zimbabwe,2021-04-12,19.0,4.0,28.0
85308,Zimbabwe,2021-04-13,23.0,1.0,31.0


In [48]:
new_cases.rename(columns={'confirmed': 'new_confirmed',
                          'deaths': 'new_deaths',
                          'recovered': 'new_recovered'}, 
                 inplace=True)

new_cases.head()

,country,date,new_confirmed,new_deaths,new_recovered
0,Afghanistan,2020-01-22,NaN,NaN,NaN
1,Afghanistan,2020-01-23,0.0,0.0,0.0
2,Afghanistan,2020-01-24,0.0,0.0,0.0
3,Afghanistan,2020-01-25,0.0,0.0,0.0
4,Afghanistan,2020-01-26,0.0,0.0,0.0


In [49]:
# merging new values on columns that are shared
world_covid_grouped = pd.merge(world_covid_grouped, new_cases, on=['country', 
                                                                   'date']).fillna(0)

In [50]:
world_covid_grouped

,date,country,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered
0,2020-01-22,Afghanistan,0,0,0,0,0.0,0.0,0.0
1,2020-01-22,Albania,0,0,0,0,0.0,0.0,0.0
2,2020-01-22,Algeria,0,0,0,0,0.0,0.0,0.0
3,2020-01-22,Andorra,0,0,0,0,0.0,0.0,0.0
4,2020-01-22,Angola,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
85305,2021-04-14,Venezuela,178094,1834,160620,15640,1122.0,19.0,1073.0
85306,2021-04-14,Vietnam,2733,35,2445,253,19.0,0.0,0.0
85307,2021-04-14,Yemen,5582,1083,2128,2371,75.0,10.0,58.0
85308,2021-04-14,Zambia,90389,1229,88241,919,171.0,0.0,97.0


In [51]:
world_covid_grouped[['new_confirmed',
                     'new_deaths',
                     'new_recovered']] = world_covid_grouped[['new_confirmed',
                                                              'new_deaths',
                                                              'new_recovered']].astype('int')

In [52]:
world_covid_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85310 entries, 0 to 85309
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           85310 non-null  datetime64[ns]
 1   country        85310 non-null  object        
 2   confirmed      85310 non-null  int64         
 3   deaths         85310 non-null  int64         
 4   recovered      85310 non-null  int64         
 5   active         85310 non-null  int64         
 6   new_confirmed  85310 non-null  int64         
 7   new_deaths     85310 non-null  int64         
 8   new_recovered  85310 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(1)
memory usage: 6.5+ MB


In [53]:
# verify if there are any negative values in the new columns
(world_covid_grouped[['new_confirmed','new_deaths','new_recovered']].values < 0).any()

True

In [54]:
# cheking some examples of negative values
print(world_covid_grouped['new_confirmed'].min())
print(world_covid_grouped['new_deaths'].min())
print(world_covid_grouped['new_recovered'].min())

-74347
-1918
-6298082


In [55]:
for col in ['new_confirmed', 'new_deaths', 'new_recovered']:
    world_covid_grouped[col] = world_covid_grouped[col].apply(func)

In [56]:
# cheking some examples to confirm the need changes
print(world_covid_grouped['new_confirmed'].min())
print(world_covid_grouped['new_deaths'].min())
print(world_covid_grouped['new_recovered'].min())

0
0
0


In [57]:
# or like this
(world_covid_grouped[['new_confirmed', 
                      'new_deaths', 
                      'new_recovered']].values < 0).any()

False

In [58]:
# save as .csv file
world_covid_grouped.to_csv(f'Data/New/world_covid_grouped_{yesterday}.csv', index=False)

#### Daily cases
Cheking the new daily covid cases

In [59]:
cols_check = world_covid_grouped.columns
cols_check

Index(['date', 'country', 'confirmed', 'deaths', 'recovered', 'active',
       'new_confirmed', 'new_deaths', 'new_recovered'],
      dtype='object')

In [60]:
# aggregate by date
daily = world_covid_grouped.groupby('date')[['confirmed', 
                                                   'deaths', 
                                                   'recovered',
                                                   'active', 
                                                   'new_confirmed', 
                                                   'new_deaths', 
                                                   'new_recovered']].sum().reset_index()

In [61]:
daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           449 non-null    datetime64[ns]
 1   confirmed      449 non-null    int64         
 2   deaths         449 non-null    int64         
 3   recovered      449 non-null    int64         
 4   active         449 non-null    int64         
 5   new_confirmed  449 non-null    int64         
 6   new_deaths     449 non-null    int64         
 7   new_recovered  449 non-null    int64         
dtypes: datetime64[ns](1), int64(7)
memory usage: 28.2 KB


In [62]:
daily.shape

(449, 8)

In [63]:
# number cases per 100 cases
daily['deaths_100Cases'] = np.round((daily['deaths'] / daily['confirmed']) * 100, 2)
daily['recovered_100Cases'] = np.round((daily['recovered'] / daily['confirmed']) * 100, 2)
daily['deaths_100recovered'] = np.round((daily['deaths'] / daily['recovered']) * 100, 2)

In [64]:
daily

,date,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered,deaths_100Cases,recovered_100Cases,deaths_100recovered
0,2020-01-22,557,17,30,510,0,0,0,3.05,5.39,56.67
1,2020-01-23,655,18,32,605,98,1,2,2.75,4.89,56.25
2,2020-01-24,941,26,39,876,286,8,7,2.76,4.14,66.67
3,2020-01-25,1433,42,42,1349,492,16,3,2.93,2.93,100.00
4,2020-01-26,2118,56,56,2006,685,14,14,2.64,2.64,100.00
...,...,...,...,...,...,...,...,...,...,...,...
444,2021-04-10,135354953,2927878,75847554,56579521,664507,12407,377111,2.16,56.04,3.86
445,2021-04-11,136045694,2936435,76370354,56738905,690741,8557,522800,2.16,56.14,3.84
446,2021-04-12,136662319,2945733,76753896,56962690,616625,9298,383542,2.16,56.16,3.84
447,2021-04-13,137441886,2959065,77230863,57251958,779567,13333,476967,2.15,56.19,3.83


In [65]:
daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 449 non-null    datetime64[ns]
 1   confirmed            449 non-null    int64         
 2   deaths               449 non-null    int64         
 3   recovered            449 non-null    int64         
 4   active               449 non-null    int64         
 5   new_confirmed        449 non-null    int64         
 6   new_deaths           449 non-null    int64         
 7   new_recovered        449 non-null    int64         
 8   deaths_100Cases      449 non-null    float64       
 9   recovered_100Cases   449 non-null    float64       
 10  deaths_100recovered  449 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(7)
memory usage: 38.7 KB


In [66]:
daily.shape

(449, 11)

In [67]:
daily.isnull().sum()

date                   0
confirmed              0
deaths                 0
recovered              0
active                 0
new_confirmed          0
new_deaths             0
new_recovered          0
deaths_100Cases        0
recovered_100Cases     0
deaths_100recovered    0
dtype: int64

In [68]:
# no. of countries
daily['num_countries'] = world_covid_grouped[world_covid_grouped['confirmed'] != 0].groupby('date')['country'].unique().apply(len).values

In [69]:
daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 449 non-null    datetime64[ns]
 1   confirmed            449 non-null    int64         
 2   deaths               449 non-null    int64         
 3   recovered            449 non-null    int64         
 4   active               449 non-null    int64         
 5   new_confirmed        449 non-null    int64         
 6   new_deaths           449 non-null    int64         
 7   new_recovered        449 non-null    int64         
 8   deaths_100Cases      449 non-null    float64       
 9   recovered_100Cases   449 non-null    float64       
 10  deaths_100recovered  449 non-null    float64       
 11  num_countries        449 non-null    int64         
dtypes: datetime64[ns](1), float64(3), int64(8)
memory usage: 42.2 KB


In [70]:
# fillna by 0
daily[['deaths_100Cases', 
             'recovered_100Cases', 
             'deaths_100recovered']] = daily[['deaths_100Cases', 
                                                    'recovered_100Cases', 
                                                    'deaths_100recovered']].fillna(0)

In [71]:
daily.sample(5)

,date,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered,deaths_100Cases,recovered_100Cases,deaths_100recovered,num_countries
335,2020-12-22,78133328,1722643,43625992,32784693,650261,14307,354926,2.20,55.84,3.95,189
121,2020-05-22,5221873,341696,2008627,2871550,106422,5247,107669,6.54,38.47,17.01,185
353,2021-01-09,89773505,1933576,49168474,38671455,760994,12781,323528,2.15,54.77,3.93,189
32,2020-02-23,78291,2467,22304,53520,323,10,452,3.15,28.49,11.06,31
317,2020-12-04,66050923,1521826,42081285,22447812,683276,12498,484872,2.30,63.71,3.62,189


In [72]:
print(daily['confirmed'].min())
print(daily['deaths'].min())
print(daily['recovered'].min())
print(daily['active'].min())
print(daily['new_deaths'].min())
print(daily['new_recovered'].min())
print(daily['deaths_100Cases'].min())
print(daily['deaths_100Cases'].min())
print(daily['recovered_100Cases'].min())
print(daily['deaths_100recovered'].min())

557
17
30
510
0
0
2.04
2.04
1.74
3.45


In [73]:
# save as .csv file
daily.to_csv(f'Data/New/world_covid_daily_{yesterday}.csv', index=False)

#### Country  data

In [74]:
# group by country
# getting latest values
country = world_covid_grouped[world_covid_grouped['date'] == max(world_covid_grouped['date'])].reset_index(drop=True)

In [75]:
country.head()

,date,country,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered
0,2021-04-14,Afghanistan,57492,2532,52022,2938,128,3,9
1,2021-04-14,Albania,128959,2331,99441,27187,207,5,538
2,2021-04-14,Algeria,118975,3141,82929,32905,176,4,116
3,2021-04-14,Andorra,12641,121,11989,531,27,0,57
4,2021-04-14,Angola,23841,557,22144,1140,144,3,29


In [76]:
country.drop('date', axis=1, inplace=True)
country.head()

,country,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered
0,Afghanistan,57492,2532,52022,2938,128,3,9
1,Albania,128959,2331,99441,27187,207,5,538
2,Algeria,118975,3141,82929,32905,176,4,116
3,Andorra,12641,121,11989,531,27,0,57
4,Angola,23841,557,22144,1140,144,3,29


In [77]:
# aggregating by countries
# group by country
country = country.groupby('country')[['confirmed',
                                      'deaths',
                                      'recovered', 
                                      'active',
                                      'new_confirmed', 
                                      'new_deaths', 
                                      'new_recovered']].sum().reset_index()

In [78]:
country.head()

,country,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered
0,Afghanistan,57492,2532,52022,2938,128,3,9
1,Albania,128959,2331,99441,27187,207,5,538
2,Algeria,118975,3141,82929,32905,176,4,116
3,Andorra,12641,121,11989,531,27,0,57
4,Angola,23841,557,22144,1140,144,3,29


In [79]:
country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   country        190 non-null    object
 1   confirmed      190 non-null    int64 
 2   deaths         190 non-null    int64 
 3   recovered      190 non-null    int64 
 4   active         190 non-null    int64 
 5   new_confirmed  190 non-null    int64 
 6   new_deaths     190 non-null    int64 
 7   new_recovered  190 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 12.0+ KB


In [80]:
# per 100 cases
country['deaths_100_cases'] = round((country['deaths']/country['confirmed']) * 100, 2)
country['recovered_100_cases'] = round((country['recovered']/country['confirmed']) * 100, 2)
country['deaths_100_recovered'] = round((country['deaths']/country['recovered']) * 100, 2)

country[['deaths_100_cases', 
         'recovered_100_cases', 
         'deaths_100_recovered']] = country[['deaths_100_cases', 
                                             'recovered_100_cases', 
                                             'deaths_100_recovered']].fillna(0)

In [81]:
country.head()

,country,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered,deaths_100_cases,recovered_100_cases,deaths_100_recovered
0,Afghanistan,57492,2532,52022,2938,128,3,9,4.40,90.49,4.87
1,Albania,128959,2331,99441,27187,207,5,538,1.81,77.11,2.34
2,Algeria,118975,3141,82929,32905,176,4,116,2.64,69.70,3.79
3,Andorra,12641,121,11989,531,27,0,57,0.96,94.84,1.01
4,Angola,23841,557,22144,1140,144,3,29,2.34,92.88,2.52


In [82]:
print(country['deaths_100_cases'].min(), country['deaths_100_cases'].max())
print(country['recovered_100_cases'].min(), country['recovered_100_cases'].max())
print(country['deaths_100_recovered'].min(), country['deaths_100_recovered'].max())

0.0 19.4
0.0 100.0
0.0 inf


In [83]:
country = country.replace([np.inf, -np.inf], 0)

In [84]:
country.isnull().sum()

country                 0
confirmed               0
deaths                  0
recovered               0
active                  0
new_confirmed           0
new_deaths              0
new_recovered           0
deaths_100_cases        0
recovered_100_cases     0
deaths_100_recovered    0
dtype: int64

In [85]:
print(country['deaths_100_cases'].min(), country['deaths_100_cases'].max())
print(country['recovered_100_cases'].min(), country['recovered_100_cases'].max())
print(country['deaths_100_recovered'].min(), country['deaths_100_recovered'].max())

0.0 19.4
0.0 100.0
0.0 931.95


In [86]:
# max(world_covid_grouped['date'])

In [87]:
# 1 week increase and % change
confirmed_today = world_covid_grouped[world_covid_grouped['date'] == max(world_covid_grouped['date'])].reset_index(drop=True)[['country',
                                                                                                                               'confirmed']]

In [88]:
confirmed_today

,country,confirmed
0,Afghanistan,57492
1,Albania,128959
2,Algeria,118975
3,Andorra,12641
4,Angola,23841
...,...,...
185,Venezuela,178094
186,Vietnam,2733
187,Yemen,5582
188,Zambia,90389


In [89]:
## data from today - the last week data
last_week = world_covid_grouped[world_covid_grouped['date'] == max(world_covid_grouped['date']) - timedelta(days=7)].reset_index(drop=True)[['country',
                                                                                                                                           'confirmed']]

In [90]:
last_week

,country,confirmed
0,Afghanistan,56873
1,Albania,127192
2,Algeria,118004
3,Andorra,12363
4,Angola,23010
...,...,...
185,Venezuela,170189
186,Vietnam,2659
187,Yemen,5047
188,Zambia,89386


In [91]:
week = pd.merge(confirmed_today, last_week, on='country', suffixes=('_today', '_last_week'))

In [92]:
week.head(2)

,country,confirmed_today,confirmed_last_week
0,Afghanistan,57492,56873
1,Albania,128959,127192


In [93]:
week.rename(columns={'confirmed_today': 'today_cases',
                     'confirmed_last_week': 'last_week_cases'},
           inplace=True)

In [94]:
week.head(4)

,country,today_cases,last_week_cases
0,Afghanistan,57492,56873
1,Albania,128959,127192
2,Algeria,118975,118004
3,Andorra,12641,12363


In [95]:
week['week_dif_change'] = week['today_cases'] - week['last_week_cases']

In [96]:
week.head()

,country,today_cases,last_week_cases,week_dif_change
0,Afghanistan,57492,56873,619
1,Albania,128959,127192,1767
2,Algeria,118975,118004,971
3,Andorra,12641,12363,278
4,Angola,23841,23010,831


In [97]:
country = pd.merge(country, week, on='country')

# getting the week diference cases percent
country['week_%_increase'] = round(country['week_dif_change'] / country['last_week_cases'] * 100, 2)
country.head()

,country,confirmed,deaths,recovered,active,new_confirmed,new_deaths,new_recovered,deaths_100_cases,recovered_100_cases,deaths_100_recovered,today_cases,last_week_cases,week_dif_change,week_%_increase
0,Afghanistan,57492,2532,52022,2938,128,3,9,4.40,90.49,4.87,57492,56873,619,1.09
1,Albania,128959,2331,99441,27187,207,5,538,1.81,77.11,2.34,128959,127192,1767,1.39
2,Algeria,118975,3141,82929,32905,176,4,116,2.64,69.70,3.79,118975,118004,971,0.82
3,Andorra,12641,121,11989,531,27,0,57,0.96,94.84,1.01,12641,12363,278,2.25
4,Angola,23841,557,22144,1140,144,3,29,2.34,92.88,2.52,23841,23010,831,3.61


In [98]:
# save as .csv file
country.to_csv(f'Data/New/country_week_latest_{yesterday}.csv', index=False)

#### Brazil data

In [99]:
world_covid_clean[world_covid_clean['country'] == 'Brazil']

,province,country,lat,lon,date,confirmed,deaths,recovered,active
30,,Brazil,-14.235,-51.9253,2020-01-22,0,0,0,0
304,,Brazil,-14.235,-51.9253,2020-01-23,0,0,0,0
578,,Brazil,-14.235,-51.9253,2020-01-24,0,0,0,0
852,,Brazil,-14.235,-51.9253,2020-01-25,0,0,0,0
1126,,Brazil,-14.235,-51.9253,2020-01-26,0,0,0,0
...,...,...,...,...,...,...,...,...,...
121686,,Brazil,-14.235,-51.9253,2021-04-10,13445006,351334,11739649,1354023
121960,,Brazil,-14.235,-51.9253,2021-04-11,13482023,353137,11878958,1249928
122234,,Brazil,-14.235,-51.9253,2021-04-12,13517808,354617,11892229,1270962
122508,,Brazil,-14.235,-51.9253,2021-04-13,13599994,358425,11975121,1266448


In [100]:
# masking
# f = temp['country'] != temp['country'].shift(1)
# temp.loc[f, 'confirmed'] = np.nan
# temp.loc[f, 'deaths'] = np.nan
# temp.loc[f, 'recovered'] = np.nan